Instructions can be found at [Implementing the Perceptron Algorithm](https://middlebury-csci-0451.github.io/CSCI-0451/assignments/blog-posts/blog-post-perceptron.html).

## Data

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

from sklearn.datasets import make_blobs

np.random.seed(12345)

n = 100
p_features = 3

X, y = make_blobs(
    n_samples=100,
    n_features=p_features - 1,
    centers=[(-1.7, -1.7), (1.7, 1.7)]
)

fig = plt.scatter(X[:, 0], X[:, 1], c=y)
xlab = plt.xlabel("Feature 1")
ylab = plt.ylabel("Feature 2")


## Algorithm

In [ ]:
%%script false --no-raise-error

from perceptron import Perceptron

p = Perceptron()
p.fit(X, y, max_steps = 1000)

In [ ]:
%%script false --no-raise-error

# score history
fig = plt.plot(p.history)
xlab = plt.xlabel("Iteration")
ylab = plt.ylabel("Accuracy")